**READ BEFORE RUNNING THE NOTEBOOK:**

In this notebook we have fine tuned 2 models, so if you are running this on colab, you must run one model section only then disconnet runtime and then rerun again the dataset loading section and the 2nd model part.
As the 2nd model, which is Llama has lower loss during training so we perform inference only on that model however, if you want to perform inference on the first model, save the model and perform inference  by configuring the path

# Dataset loading

In [ ]:
!git clone https://github.com/MuhammadBinUsman03/bankbot-llm.git /content/repo

fatal: destination path '/content/repo' already exists and is not an empty directory.


## Importing libraries

In [ ]:
!pip install --upgrade pip
!pip install transformers datasets accelerate peft bitsandbytes trl

## configuring paths

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig

data_dir = "/content/repo/bankbot-llm/Dataset/Instruction Tuning Dataset"
files = [
  f"{data_dir}/BankProducts_FineTuning.json",
  f"{data_dir}/finetune_data.json"
]

## Dataset preparation

In [ ]:
raw_ds = load_dataset("json", data_files={"train": files}, field=None)["train"]
print(f"Total examples: {len(raw_ds)}")

Total examples: 319


In [ ]:
def flatten(example):
    prompt_txt = " ".join(m["content"] for m in example["prompt"]).strip()
    completion_txt = example["completion"][-1]["content"].strip()
    return {"prompt_text": prompt_txt, "completion_text": completion_txt}

ds = raw_ds.map(flatten, remove_columns=["prompt","completion"])
print(ds[0])

{'prompt_text': 'I would like to open an account with my son, do u have any product for kids?', 'completion_text': 'Yes our product is Little Champs Account. It is designed specifically for minors (individuals below the age of 18 years). A child requires the help of a parental/legal guardian to open this account and avail its facilities. Little Champs get a Debit Card and chequebook which is free the first time'}


In [ ]:
split = ds.train_test_split(test_size=0.05, seed=42)
train_ds, valid_ds = split["train"], split["test"]

## Shared model attributes

In [ ]:
# Step 7: Tokenization function (same for both models)
def tokenize_fn(exs, tokenizer):
    in_ids = tokenizer(exs["prompt_text"], truncation=True, max_length=512).input_ids
    out_ids= tokenizer(exs["completion_text"], truncation=True, max_length=512).input_ids
    return {"input_ids": in_ids, "labels": out_ids}

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `llm_project` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm_project`

In [ ]:
# Step 8: Define SFT training args (shared)
training_args = SFTConfig(
    output_dir="outputs",
    num_train_epochs=20,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    logging_steps=20,
    eval_steps=200,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=True,
)

In [ ]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Models

## Model 1: Deepseek- variant with 1.5b params

### loading model

In [ ]:
# Step 5: Load & wrap DeepSeek‑Qwen model for LoRA
model1_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer1 = AutoTokenizer.from_pretrained(model1_id)
model1 = AutoModelForCausalLM.from_pretrained(model1_id, device_map="auto", torch_dtype="auto")

model1 = get_peft_model(model1, peft_config)
model1.print_trainable_parameters()  # check LoRA parameters

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

trainable params: 1,089,536 || all params: 1,778,177,536 || trainable%: 0.0613


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'meta-llama/Llama-3.2-3B-Instruct' to 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


### prepraring tokenized dataset for model

In [ ]:
tok_train1 = train_ds.map(lambda x: tokenize_fn(x, tokenizer1), batched=True, remove_columns=train_ds.column_names)
tok_valid1 = valid_ds.map(lambda x: tokenize_fn(x, tokenizer1), batched=True, remove_columns=valid_ds.column_names)

Map:   0%|          | 0/303 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [ ]:
trainer1 = SFTTrainer(
    model=model1,
    args=training_args,
    train_dataset=tok_train1,
    eval_dataset=tok_valid1,
    peft_config=peft_config,
)

Truncating train dataset:   0%|          | 0/303 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/16 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
import math

### Fine tuning model 1

In [ ]:
# Step 10: Fine‑tune both models
print("▶️ Training DeepSeek‑Qwen‑1.5B …")
trainer1.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


▶️ Training DeepSeek‑Qwen‑1.5B …


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tsardar-bese21seecs (tsardar-bese21seecs-national-university-of-science-and-t) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
20,6.402900
40,6.269200
60,6.126000
80,5.967900
100,5.851600
120,5.741800
140,5.667000
160,5.608000
180,5.579000


TrainOutput(global_step=180, training_loss=5.912606472439236, metrics={'train_runtime': 271.0763, 'train_samples_per_second': 22.355, 'train_steps_per_second': 0.664, 'total_flos': 905007881625600.0, 'train_loss': 5.912606472439236})

### Evaluating model 1

In [ ]:
res1 = trainer1.evaluate()

ppl1 = math.exp(res1["eval_loss"])

print(f"✅ DeepSeek‑Qwen PPL: {ppl1:.2f}")

✅ DeepSeek‑Qwen PPL: 249.68


### Save model weights

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Create a folder in Drive for your model
save_path = "/content/drive/MyDrive/qwen-finetuned"
trainer1.model.save_pretrained(save_path)
tokenizer1.save_pretrained(save_path)

print(f"✅ Model & tokenizer saved to {save_path}")

✅ Model & tokenizer saved to /content/drive/MyDrive/qwen-finetuned


## Model 2: Llama varaint with 3b params

### loading model

In [ ]:
# Step 6: Load & wrap Llama‑3.2‑3B for LoRA
model2_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer2 = AutoTokenizer.from_pretrained(model2_id)

model2 = AutoModelForCausalLM.from_pretrained(model2_id, device_map="auto", torch_dtype="auto")

model2 = get_peft_model(model2, peft_config)
model2.print_trainable_parameters()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

trainable params: 2,293,760 || all params: 3,215,043,584 || trainable%: 0.0713


### preparing dataset for model 2

In [ ]:
tok_train2 = train_ds.map(lambda x: tokenize_fn(x, tokenizer2), batched=True, remove_columns=train_ds.column_names)
tok_valid2 = valid_ds.map(lambda x: tokenize_fn(x, tokenizer2), batched=True, remove_columns=valid_ds.column_names)

Map:   0%|          | 0/303 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [ ]:
trainer2 = SFTTrainer(
    model=model2,
    args=training_args,
    train_dataset=tok_train2,
    eval_dataset=tok_valid2,
    peft_config=peft_config,

)

Truncating train dataset:   0%|          | 0/303 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/16 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


###

### Fine tuning model 2

In [ ]:
import math

In [ ]:
print("▶️ Training Llama‑3B …")
trainer2.train()

▶️ Training Llama‑3B …


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tsardar-bese21seecs (tsardar-bese21seecs-national-university-of-science-and-t) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
20,5.212300
40,4.997900
60,4.752600
80,4.501500
100,4.283900
120,4.100900
140,3.944300
160,3.844500
180,3.799500


TrainOutput(global_step=180, training_loss=4.381936264038086, metrics={'train_runtime': 322.5784, 'train_samples_per_second': 18.786, 'train_steps_per_second': 0.558, 'total_flos': 1646720709574656.0, 'train_loss': 4.381936264038086})

### Evaluating model 2

In [ ]:
res2 = trainer2.evaluate()

ppl2 = math.exp(res2["eval_loss"])

print(f"✅ Llama‑3B    PPL: {ppl2:.2f}")


✅ Llama‑3B    PPL: 38.45


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
save_path2 = "/content/drive/MyDrive/llama3b-finetuned"
trainer2.model.save_pretrained(save_path2)
tokenizer2.save_pretrained(save_path2)

print(f"✅ Llama model & tokenizer saved to {save_path2}")

✅ Llama model & tokenizer saved to /content/drive/MyDrive/llama3b-finetuned


# Note

Seeing the results of both model Llama performs better on fine tuning so we will move forward with that and perform inference only on Llama one

# Inference part

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

In [ ]:
import torch

In [ ]:
inference_path = "/content/drive/MyDrive/llama3b-finetuned"
tokenizer = AutoTokenizer.from_pretrained(inference_path)
model     = AutoModelForCausalLM.from_pretrained(
    inference_path,
    device_map="auto",
    torch_dtype=torch.float16
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Assume valid_ds has columns "prompt_text" and "completion_text"
prompts = valid_ds["prompt_text"]
references = valid_ds["completion_text"]

In [ ]:
def generate_batch(model, tokenizer, texts, max_new_tokens=100, device="cuda"):
    model.to(device)
    preds = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt").to(device)
        out = model.generate(**inputs, max_new_tokens=max_new_tokens)
        gen = tokenizer.decode(out[0], skip_special_tokens=True)
        # Remove the prompt from the output if the model echoes it
        gen = gen[len(text):].strip()
        preds.append(gen)
    return preds

# Generate for both models
preds = generate_batch(model, tokenizer, prompts)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [ ]:
!pip install evaluate
!pip install rouge_score
!pip install bert_score

In [ ]:
import evaluate
import torch

In [ ]:
# 3. Compute automatic metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

In [ ]:
# BLEU expects references as list of lists
bleu_score = bleu.compute(predictions=preds, references=[[r] for r in references])

# ROUGE returns a dict of scores
rouge_score = rouge.compute(predictions=preds, references=references)

# BERTScore returns precision/recall/f1 lists; take the mean F1
bertscore = bertscore.compute(predictions=preds, references=references, lang="en")["f1"]
avg_bertscore = sum(bertscore) / len(bertscore)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def print_metric(blue_score, rouge_score, bertscore, avg_bertscore):
  print(f"Bleu: {blue_score}")
  print(f"Rouge: {rouge_score}")
  print(f"Bert Score: {bertscore}")
  print(f"Average Bert Score: {avg_bertscore}")

In [ ]:
print ("Model: llama3b-finetuned results")
print_metric(bleu_score, rouge_score, bertscore, avg_bertscore)

Model: llama3b-finetuned results
Bleu: {'bleu': 0.014566435218958854, 'precisions': [0.07263751763046544, 0.01783166904422254, 0.007936507936507936, 0.004379562043795621], 'brevity_penalty': 1.0, 'length_ratio': 4.43125, 'translation_length': 1418, 'reference_length': 320}
Rouge: {'rouge1': np.float64(0.1082699661418545), 'rouge2': np.float64(0.028128958886832887), 'rougeL': np.float64(0.08159219070789164), 'rougeLsum': np.float64(0.08996909976826693)}
Bert Score: [0.8112339973449707, 0.779411792755127, 0.843319296836853, 0.8624247908592224, 0.7643372416496277, 0.8227237462997437, 0.8430028557777405, 0.8565472364425659, 0.7800590395927429, 0.8615150451660156, 0.7819951176643372, 0.789619505405426, 0.8522619605064392, 0.7930500507354736, 0.8479657173156738, 0.8486303091049194]
Average Bert Score: 0.8211311064660549


In [ ]:
examples = [
    "How much can I transfer per day via mobile banking?",
    "What documents are required to open a savings account?"
]

for text in examples:
    # Llama generation
    inputs = tokenizer(text, return_tensors="pt").to(model2.device)
    out = model.generate(**inputs, max_new_tokens=100)
    print("Llama→", tokenizer.decode(out[0], skip_special_tokens=True))
    print("-" * 60)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Llama→ How much can I transfer per day via mobile banking??
For more information, please contact our customer service team at +880 1717 1717 or [support@bracbank.com](mailto:support@bracbank.com) or visit our website at [www.bracbank.com](http://www.bracbank.com). 

For Mobile Banking, the daily transfer limit is Tk 50,000. However, this limit may vary depending on the type of account and the user's account balance. Please check your account details and the
------------------------------------------------------------
Llama→ What documents are required to open a savings account? to open a savings account? Here are some of the documents that you need to have for opening a savings account: 1. A valid identification document such as a passport, driver’s license, or national ID card. 2. Proof of address such as a utility bill or bank statement. 3. Social Security number or Taxpayer Identification Number (TIN) for US citizens. 4. Proof of income, such as a pay stub or W-2 form. 5. For
------